In [17]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [23]:
# Zeigt alle Patienten, die von einem bestimmten Arzt behandelt werden

query = """
SELECT DISTINCT
    a.vorname || ' ' || a.nachname AS arzt_name,
    p.vorname || ' ' || p.nachname AS patient_name,
    p.geburtsdatum,
    b.beschreibung AS behandlung
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE a.arzt_id = 1
ORDER BY patient_name;
"""
cur.execute(query)

columns = [d[0] for d in cur.description]
print(columns)
for row in cur.fetchall():
    print(row)


['arzt_name', 'patient_name', 'geburtsdatum', 'behandlung']
('Sascha Bader', 'Afrim Gerber', datetime.date(2015, 11, 30), 'Blutdruckeinstellung')
('Sascha Bader', 'Alyssa Stauffer', datetime.date(2014, 6, 12), 'Wundversorgung')
('Sascha Bader', 'Ana Gross', datetime.date(1933, 11, 3), 'Diabetes Verlaufskontrolle')
('Sascha Bader', 'Anaïs Gross', datetime.date(1970, 3, 27), 'Wundversorgung')
('Sascha Bader', 'Andrew Käser', datetime.date(1984, 6, 3), 'Schmerztherapie')
('Sascha Bader', 'Anna Bühlmann', datetime.date(1985, 2, 23), 'Präoperative Abklärung')
('Sascha Bader', 'Anne-Sophie Schär', datetime.date(2014, 7, 19), 'Appendektomie Nachkontrolle')
('Sascha Bader', 'Ava Stadelmann', datetime.date(1935, 12, 26), 'Blutdruckeinstellung')
('Sascha Bader', 'Carole Siegrist', datetime.date(1980, 12, 16), 'Diabetes Verlaufskontrolle')
('Sascha Bader', 'Christof Rüegg', datetime.date(1979, 6, 22), 'Herz-Kreislauf Check')
('Sascha Bader', 'Claudio Scherrer', datetime.date(1941, 7, 1), 'Postope

In [24]:
# Zeigt alle Events an einem bestimmten Datum (inkl. Raum & Beschreibung)

query = """
SELECT
    e.event_id,
    e.datum,
    e.uhrzeit,
    e.beschreibung,
    r.art AS raum_art,
    s.name AS station
FROM event e
JOIN raum r ON e.raumid = r.raumnummer
JOIN station s ON r.station_id = s.station_id
WHERE e.datum = DATE '2025-12-12'
ORDER BY e.uhrzeit;
"""
cur.execute(query)

columns = [d[0] for d in cur.description]
print(columns)
for row in cur.fetchall():
    print(row)


['event_id', 'datum', 'uhrzeit', 'beschreibung', 'raum_art', 'station']
(214, datetime.date(2025, 12, 12), datetime.time(0, 4, 28), 'Pflegevisite', 'Untersuchungsraum', 'Intensivstation')
(50, datetime.date(2025, 12, 12), datetime.time(0, 29, 31), 'Laborbesprechung', 'Operationssaal', 'Innere Medizin')
(35, datetime.date(2025, 12, 12), datetime.time(0, 45, 36), 'Laborbesprechung', 'Untersuchungsraum', 'Pädiatrie')
(75, datetime.date(2025, 12, 12), datetime.time(0, 54, 29), 'Pflegevisite', 'Operationssaal', 'Chirurgie')
(282, datetime.date(2025, 12, 12), datetime.time(0, 59, 4), 'Laborbesprechung', 'Operationssaal', 'Chirurgie')
(264, datetime.date(2025, 12, 12), datetime.time(1, 4, 51), 'Nachkontrolle', 'Operationssaal', 'Notfall')
(304, datetime.date(2025, 12, 12), datetime.time(1, 5, 40), 'Pflegevisite', 'Untersuchungsraum', 'Innere Medizin')
(99, datetime.date(2025, 12, 12), datetime.time(1, 13, 15), 'Therapieanpassung', 'Operationssaal', 'Radiologie')
(268, datetime.date(2025, 12, 

In [20]:
# Query 2: Zeigt pro Station, wie viele Betten aktuell frei sind.

query = """
SELECT
    s.name AS station,
    COUNT(b.bett_id) AS freie_betten
FROM bett b
JOIN station s ON b.station_id = s.station_id
WHERE b.status = 'frei'
GROUP BY s.name
ORDER BY freie_betten DESC;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['station', 'freie_betten']
('Chirurgie', 2)
('Innere Medizin', 2)
('Intensivstation', 2)
('Notfall', 2)
('Pädiatrie', 2)
('Radiologie', 2)


In [21]:
# Query 4: Zeigt alle Patienten mit den Medikamenten,
# die ihnen im Rahmen von Events verabreicht wurden.

query = """
SELECT
    p.vorname || ' ' || p.nachname AS patient_name,
    m.name AS medikament,
    m.wirkstoff
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
JOIN event e ON b.behandlungs_id = e.behandlung_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
ORDER BY patient_name;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['patient_name', 'medikament', 'wirkstoff']
('Adrian Schärer', 'Ibuprofen', 'Ibuprofen')
('Adrian Schärer', 'Amoxicillin', 'Amoxicillin')
('Adrian Schärer', 'Aspirin', 'Acetylsalicylsäure')
('Adrian Schärer', 'Ibuprofen', 'Ibuprofen')
('Afrim Gerber', 'Paracetamol', 'Paracetamol')
('Afrim Gerber', 'Amoxicillin', 'Amoxicillin')
('Afrim Gerber', 'Amoxicillin', 'Amoxicillin')
('Afrim Gerber', 'Insulin', 'Insulin')
('Agnes Grob', 'Amoxicillin', 'Amoxicillin')
('Agnes Grob', 'Aspirin', 'Acetylsalicylsäure')
('Agnes Grob', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Ibuprofen', 'Ibuprofen')
('Albina Schmidt', 'Insulin', 'Insulin')
('Alfredo Fischer', 'Insulin', 'Insulin')
('Alfredo Fischer', 'Aspirin', 'Acetylsalicylsäure')
('Alyssa Stauffer', 'Ibuprofen', 'Ibuprofen')
('Alyssa Stauffer', 'Insulin', 'Insulin')
('Alyssa Stauffer', 'Paracetamol', 'Paracetamol')
('Alyssa Stauffer', 'Amoxicillin', 'Amoxicillin')
('Alyssa Stauffer', 'Insulin', 'Insul

In [22]:
# Query 5: Zeigt alle Events des heutigen Tages,
# sortiert nach Raum und Uhrzeit (z. B. für Tagesplanung).

query = """
SELECT
    r.art AS raum_art,
    r.raumnummer,
    e.uhrzeit,
    p.vorname || ' ' || p.nachname AS patient_name
FROM event e
JOIN raum r ON e.raumid = r.raumnummer
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE e.datum = CURRENT_DATE
ORDER BY r.raumnummer, e.uhrzeit;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['raum_art', 'raumnummer', 'uhrzeit', 'patient_name']
('Operationssaal', 1, datetime.time(0, 54, 29), 'Kathleen Portmann')
('Operationssaal', 1, datetime.time(0, 59, 4), 'Pierina Kaufmann')
('Operationssaal', 1, datetime.time(6, 28, 45), 'Karine Ferrari')
('Operationssaal', 1, datetime.time(14, 44, 25), 'Jeannette Maurer')
('Operationssaal', 1, datetime.time(15, 28, 47), 'Afrim Gerber')
('Operationssaal', 1, datetime.time(17, 8, 6), 'Elena Bolliger')
('Untersuchungsraum', 2, datetime.time(5, 1, 32), 'Ilaria Wegmann')
('Untersuchungsraum', 2, datetime.time(7, 48, 21), 'Rosalie Schärer')
('Untersuchungsraum', 2, datetime.time(10, 33, 30), 'Cristina Sutter')
('Untersuchungsraum', 2, datetime.time(15, 35, 5), 'Albina Schmidt')
('Untersuchungsraum', 2, datetime.time(15, 47, 24), 'Nebojsa Schär')
('Untersuchungsraum', 2, datetime.time(23, 1, 13), 'Raymond Meister')
('Operationssaal', 3, datetime.time(0, 29, 31), 'Emil Steinmann')
('Operationssaal', 3, datetime.time(5, 32, 44), 'Ryan Rüegg')
